### Behaviour dimensionalitet
136200 readings x 18 features -> 5 readings svarer til 1 epoch fordi det er lidt mærkeligt stillet op..
136200/93/1500 - > cirka 300 epochs per pers med nogle slettede

### FG dimensionalitet
92 x 19 features.. Der er en person der mangler her

### Force dimensionalitet
4.5 mil x 11 features.
25 readings per sekund i force. 4 sekunder. 5 Forskellige parametre, P1, P2, P3, Joint force og Target Force


### EEG data
92 participants, op til 300 epochs.
92 x 300 -> Inspicerer man 1 participant får man 900K readings, da der er 500Hz refreshing og 6 sekunders tid [-0.5s,5.5s]

Så hvad passer sammen?

EEG data + FG dimensionalitet passer umiddelbart godt sammen

Jeg ved ikke om det umiddelbart er en god ide at samle det til 1 stor dataframe til at starte med

Vi skal have dette til at passe med labrams patch størrelser


In [7]:
import mne
import pandas as pd
import os
import pickle

# Define paths
folder_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData"
overview_path = "/Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/FG_overview_df_v2.pkl"

# Load overview data
with open(overview_path, "rb") as f:
    overview_df = pickle.load(f)

# Initialize lists to store data
all_data = []

# Get list of all .fif files
fif_files = [f for f in os.listdir(folder_path) if f.endswith('.fif')]

t = 0
# Process each file
for fif_file in fif_files:
    t += 1
    if t == 11:
        break
    # Extract participant ID from filename
    participant_id = fif_file.split('_')[0]
    triad_id = int(participant_id[:3])
    
    # Get participant overview data
    participant_overview = overview_df[overview_df['Exp_id'] == participant_id].iloc[0]
    
    # Load epochs
    epochs = mne.read_epochs(os.path.join(folder_path, fif_file), preload=True)
    eeg_data = epochs.get_data()
    
    #normalize like they do to 0.1 mV
    eeg_data = eeg_data * 10000
    

    # Get event IDs to condition mapping
    event_id_to_condition = {v: k for k, v in epochs.event_id.items()}
    
    # For each epoch
    for epoch_idx in range(len(epochs)):
        # Get condition for this epoch
        event_id = epochs.events[epoch_idx][2]
        condition = event_id_to_condition[event_id]
        trial_number = epoch_idx + 1  # 1-based trial number
        
        # Create row for each epoch
        row_data = {
            # Participant information
            'participant_id': participant_id,
            'triad_id': triad_id,
            'subject_id': participant_overview['Subject_id'],
            'friend_status': participant_overview['Friend_status'],
            'age': participant_overview['Age'],
            'gender': participant_overview['Gender'],
            'class_friends': participant_overview['Class_friends'],
            'class_close_friends': participant_overview['Class_close_friends'],
            'friends': participant_overview['Friends'],
            
            # Trial information
            'epoch_idx': epoch_idx,
            'trial_number': trial_number,
            'condition': condition,
            
            # Device information
            'eeg_device': participant_overview['EEG_device'],
            'force_device': participant_overview['Force_device'],
            'force_port': participant_overview['Force_port'],
            
            # EEG data
            'eeg_data': eeg_data[epoch_idx]
        }
        all_data.append(row_data)

# Create DataFrame
combined_df = pd.DataFrame(all_data)

# Print basic information about the DataFrame
print("DataFrame shape:", combined_df.shape)
print("\nColumns:", combined_df.columns.tolist())
print("\nParticipants:", combined_df['participant_id'].nunique())
print("Total epochs:", len(combined_df))

Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/326A_FG_preprocessed-epo.fif ...


    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/311A_FG_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
300 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG_Data/PreprocessedEEGData/318B_FG_preprocessed-epo.fif ...
    Found the data of interest:
        t =    -500.00 ...    5498.00 ms
        0 CTF compensation matrices available
Not setting metadata
298 matching events found
No baseline correction applied
0 projection items activated
Reading /Users/maltelau/Desktop/LaBraM-MMDTU/LaBraM-MMDTU/DTUDATA/FG